# 図表を作成する
このノートブックではコードセルを使用して図表を作成することが可能です。<br>
ノートブックは図表ごとに作成し、図表と元データの関連性がわかるようにノートブックに明記した上で図表と共に管理してください。<br>
<span style="color:red">※このノートブックを編集した際は必ず保存してから終了してください。</span>

## 入力データを表示する
グラフの作成に用いるデータを読み込みます。<br>
「論拠データを表示する」セルを実行して表示した論拠データのフォルダから読み込ませたいデータを選択し、「ファイルを読み込む」セルにファイル名を書き込んでください。<br>
必要なファイル名を書き込んだ後、セルを実行することでファイルを読み込むことが可能です。<br>
ファイルは複数選択することも可能です。<br>
<br>
グラフの作成には論拠データとして保存したものを使用してください。<br>
論拠データとして保存していないデータを使用したい場合はデータを追加してください。<br>
現在、読み込み可能なデータの形式はcsv、excel、textのみとなっています。<br>
それ以外の形式のファイルを読み込む場合はコードセルを修正してください。<br>

In [ ]:
# 論拠データを表示する
import os
import panel as pn
from IPython.display import display
from IPython.core.display import Javascript
from library.utils.access import open_data_folder

folder_name = 'argument_data'
button_width = 500

button = open_data_folder(os.path.abspath('__file__'))

pn.extension()
display(pn.pane.HTML(button, width=button_width))
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
# ファイルを読み込む
import os
from IPython.display import display, Javascript

import pandas as pd
import matplotlib.pyplot as plt

from library.utils.config import message as msg_config
from library.utils.setting import get_data_dir
from library.task_director import TaskDirector
from library.utils.widgets import MessageBox

# ここに読み込むファイル名を書く(例：load_files =["sample.text", "sample2.csv"])
load_files =[]

class LoadFile(TaskDirector):

    def __init__(self, working_path: str):
        """DataSaver コンストラクタメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        self.working_path = working_path
        notebook_name = self.get_notebook_name()
        super().__init__(self.working_path, notebook_name)

    def get_notebook_name(self):
        script = """
        var kernel = Jupyter.notebook.kernel;
        var notebook_name = Jupyter.notebook.notebook_name;
        kernel.execute("notebook_name = '" + notebook_name + "'")
        """
        display(Javascript(script))
        return notebook_name

    @TaskDirector.task_cell("2")
    def load_file(self):

        self.doing_task()

        self.load_error_message = MessageBox()
        self.load_error_message.width = 900

        if not load_files:
            self.load_error_message.update_warning(msg_config.get('figure_template', 'file_not_exist'))
            display(self.load_error_message)
            return

        dir_name = os.path.join(get_data_dir(self.working_path), 'argument_data')
        error_files = []

        for index, file_name in enumerate(load_files):
            file_path = os.path.join(dir_name, file_name)
            if os.path.exists(file_path):
                extension = os.path.splitext(file_name)[1]

                if extension == ".csv" or ".txt":
                    f"df:{file_name}" = pd.read_csv(file_path)
                    display(f"df:{file_name}")
                elif extension == "xlsx":
                    f"df:{file_name}" = pd.read_excel(file_path) #特定のシートを読み込みたい場合はsheet_name = "")を引数に加える
                    display(f"df:{file_name}")

                else:
                    error_files.append(file_name)

            else:
                error_files.append(file_name)

        if error_files:
            self.load_error_message.update_warning(msg_config.get('figure_template', 'file_error') + error_files)
            display(self.load_error_message)

        self.done_task()
        display(Javascript('IPython.notebook.save_checkpoint();'))

LoadFile(os.path.abspath('__file__')).load_file()


## 図表を作成する
論拠データを使用して図表の作成を行ってください。

### 1. 表を作成する
このセルはあらかじめンプルの表が生成されるように記述されています。<br>
表を作成する際はサンプルのコードを書き換えて使用してください。<br>
サンプルの表の作成に使用しているライブラリの使い方を知りたい場合は[こちら](https://pandas.pydata.org/)をクリックしてください。<br>
セルを実行することで作成した表を出力することができます。

In [ ]:
# コードセルで作成する

#使用するデータフレーム(出力されたデータフレームの名前、df:{ファイル名}を入力してください)
df1 = ""

df1[['Column1', 'Column2']]

display(df1)

#### 作成した表を保存する<br>
作成した表の保存を行います。<br>
コードセルに保存するファイル名を書き込み、実行してください。<br>
作成した表はノートブックと同じフォルダに保存されます。<br>
また、表は画像ファイル(png)として保存されます。<br>
異なる形式で出力したい場合はセルのコードを書き換えて下さい。


In [ ]:
# 表を保存する
#保存する際の名前を入力してください(例："sample.png")
table_name = ""

# matplotlibのFigureとAxesを設定
fig, ax = plt.subplots(figsize=(8, 4))  # 表のサイズを指定

# 表をプロット（データフレームをmatplotlibのtable形式で表示）
ax.axis('tight')
ax.axis('off')  # 軸は非表示にする

# データフレームをmatplotlibのtableに変換して表示
table = ax.table(cellText=df1.values, colLabels=df1.columns, loc='center')

# 画像として保存
plt.savefig(table_name, format='png', bbox_inches='tight', dpi=300)

### 2. グラフを作成する
このセルはあらかじめサンプルの棒グラフが生成されるようになっています。<br>
グラフを作成する際はサンプルのコードを書き換えて使用してください。<br>
他のグラフを使用したり見た目を変更したい場合はライブラリの使い方を確認してください。<br>
サンプルグラフの作成に使用しているライブラリの使い方を知りたい場合は[こちら]( https://matplotlib.org/)をクリックしてください。<br>
セルを実行することで作成したグラフを出力することができます。

In [ ]:
# コードセルで作成する
#使用するデータフレーム(出力されたデータフレームの名前、df:{ファイル名}を入力してください)
df1 = ""

# 'Name'をx軸、'Age'をy軸にして棒グラフを作成
df1.plot(kind='bar', x='Name', y='Age', legend=False)

# タイトルと軸ラベルを追加
plt.title('Age of People')
plt.xlabel('Name')
plt.ylabel('Age')

# グラフを表示
plt.show()

#### 作成したグラフを保存する<br>
作成したグラフの保存を行います。<br>
コードセルに保存するファイル名を書き込み、実行してください。<br>
作成したグラフはノートブックと同じフォルダに保存されます。<br>
また、グラフは画像ファイル(png)として保存されます。<br>
異なる形式で出力したい場合はセルのコードを書き換えて下さい。


In [ ]:
# グラフを保存する
#保存する際の名前を入力してください(例："sample.png")
graph_name = ""

plt.savefig(graph_name, format='png', bbox_inches='tight', dpi=300)